notebook to make price predictions

uses code from the buildSegments sections which do autoregression

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import sklearn as sk
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

/Users/pauldefusco/anaconda2/envs/py27/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
master = pd.read_parquet('production_data/processed_data/master.parquet',engine='fastparquet')

In [ ]:
import statsmodels.api as sm

In [ ]:
x = master['price_y']

In [ ]:
x.index = master['date']

In [ ]:
x.index = pd.DatetimeIndex(x.index,start='0', end='lngth')

In [ ]:
model = sm.tsa.ARMA(x, (3, 1))
result = model.fit(maxiter=1000, method='mle', solver='cg')
result.summary()

In [ ]:
def mean_forecast_err(y, yhat):
    return y.sub(yhat).mean()

In [ ]:
def embed_data(x, steps):
    n = len(x)
    xout = np.zeros((n - steps, steps))
    yout = x[steps:]
    for i in np.arange(steps, n):
        xout[i - steps] = x[i-steps:i]
    return xout, yout

In [ ]:
from sklearn import svm

In [ ]:
model = svm.SVR(kernel='linear', C=1)

In [ ]:
xs = master['price_y'].head(400)

In [ ]:
x, y = embed_data(xs, 1)

In [ ]:
model.fit(x, y)

In [ ]:
model.predict(x)

In [ ]:
print metrics.mean_squared_error(model.predict(x), y)
print np.linalg.norm(model.predict(x) - y)**2 / np.linalg.norm(y)**2

In [ ]:
train = xs[:72]
test = xs[72:]
xtrain, ytrain = embed_data(train, 1)
xtest, ytest = embed_data(test, 1)

In [ ]:
m = svm.SVR(kernel='rbf', C=1, gamma=0.1)
m.fit(xtrain, ytrain)

In [ ]:
print 'R2 for the prediction'
R2 = m.score(xtest,ytest)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.recurrent import LSTM

In [ ]:
xs = master[master['bed_bath_grp']==40]['price_y']
x = xs[:-1]; y = xs[1:]

In [ ]:
xtrain = x.reshape(len(x), 1, 1)
ytrain = y.reshape(len(y), 1)

In [ ]:
bs = 1
model = Sequential()
model.add(LSTM(10, batch_input_shape=(bs, 1, 1), stateful=True))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
for i in range(3):
    model.fit(xtrain, ytrain, epochs=1, batch_size=bs, shuffle=False)
    model.reset_states()

In [ ]:
model.reset_states()

In [ ]:
yhat = model.predict(xtrain, batch_size=bs)

In [ ]:
start=1800; stop=2000
plt.figure(figsize=(15,10))
plt.plot(yhat[start:stop], '-', ytrain[start:stop], 'r-')

In [ ]:
model.nmse(yhat, ytrain)